In [53]:
import pickle
from revolve2.modular_robot import ModularRobot
from revolve2.modular_robot.brain.cpg import BrainCpgNetworkStatic
from revolve2.modular_robot.brain.cpg import BrainCpgNSEnvironmentalControl
from revolve2.modular_robot.brain.cpg._make_cpg_network_structure_neighbor import (
    active_hinges_to_cpg_network_structure_neighbor,
)
from revolve2.modular_robot_physical import Config

import numpy as np
from revolve2.modular_robot.body.v1 import *
from revolve2.modular_robot.body import RightAngles
from revolve2.modular_robot.body.base import ActiveHinge
from revolve2.modular_robot_physical import UUIDKey
import math

In [ ]:
darwinism_conf.pickle

In [54]:
CONTROL_FREQUENCY = 30 # 20
RUN_DURATION = 120

In [55]:
def darwinism() -> tuple[BodyV1, list[ActiveHingeV1]]:
    body = BodyV1()
    h1 = body.core_v1.front = ActiveHingeV1(RightAngles.DEG_90)
    h2 = body.core_v1.front.attachment = ActiveHingeV1(RightAngles.DEG_270)
    h3 = body.core_v1.front.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)

    h4 = body.core_v1.right = ActiveHingeV1(RightAngles.DEG_90)
    h5 = body.core_v1.right.attachment = ActiveHingeV1(RightAngles.DEG_270)
    h6 = body.core_v1.right.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)

    h7 = body.core_v1.back = ActiveHingeV1(RightAngles.DEG_90)
    h8 = body.core_v1.back.attachment = ActiveHingeV1(RightAngles.DEG_270)
    h9 = body.core_v1.back.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)
    hinges = [h1,h2,h3,h4,h5,h6,h7,h8,h9]
    return body, hinges


def lamarckism() -> tuple[BodyV1, list[ActiveHingeV1]]:
    body = BodyV1()
    h1 = body.core_v1.front = ActiveHingeV1(RightAngles.DEG_0)
    h2 = body.core_v1.front.attachment = ActiveHingeV1(RightAngles.DEG_0)
    h3 = body.core_v1.front.attachment.attachment = ActiveHingeV1(RightAngles.DEG_0)
    
    h4 = body.core_v1.right = ActiveHingeV1(RightAngles.DEG_90)
    h5 = body.core_v1.right.attachment = ActiveHingeV1(RightAngles.DEG_270)
    

    h6 = body.core_v1.back = ActiveHingeV1(RightAngles.DEG_90)
    h7 = body.core_v1.back.attachment = ActiveHingeV1(RightAngles.DEG_270)

    h8 = body.core_v1.left = ActiveHingeV1(RightAngles.DEG_90)
    h9 = body.core_v1.left.attachment = ActiveHingeV1(RightAngles.DEG_270)
    
    hinges =  [h1,h2,h3,h4,h5,h6,h7,h8,h9]
    return body, hinges

## Configure Darwinism


In [56]:
with open("brains_pickle/darw_brain.pickle", "rb") as f:
    brain_params = pickle.load(f)

In [57]:
body, hinges = darwinism()
pins = [11,7,6,13,10,25,17,27,23]

hinge_mapping = {UUIDKey(hinge): pin for hinge, pin in zip(hinges, pins)}
initial_hinge_positions = {UUIDKey(hinge): 0.0 for hinge in hinges}
inverse_servos = {13:True}

initial_state, _, weight_matrix, _ = brain_params

weight_matrix = weight_matrix / np.max(np.abs(weight_matrix))
(
    cpg_network_structure,
    output_mapping,
) = active_hinges_to_cpg_network_structure_neighbor(hinges)
# initial_state = cpg_network_structure.make_uniform_state(0.5 * math.sqrt(2))

brain = BrainCpgNetworkStatic(
    initial_state=initial_state,
    weight_matrix=weight_matrix,
    output_mapping=output_mapping,
)

robot = ModularRobot(body=body, brain=brain)

In [58]:
config = Config(
    modular_robot=robot,
    hinge_mapping=hinge_mapping,
    initial_hinge_positions=initial_hinge_positions,
    run_duration=RUN_DURATION,
    control_frequency=CONTROL_FREQUENCY,
    inverse_servos=inverse_servos,
)

with open("darwinism_conf.pickle", "wb") as f:
    pickle.dump(config, f)

## Configure Lamarcism

In [44]:
with open("brains_pickle/lamarc_brain.pickle", "rb") as f:
    brain_params = pickle.load(f)

In [45]:
body, hinges = lamarckism()
pins = [17, 27, 23, 6, 13, 11, 7, 10, 25] 

hinge_mapping = {UUIDKey(hinge): pin for hinge, pin in zip(hinges, pins)}
initial_hinge_positions = {UUIDKey(hinge): 0.0 for hinge in hinges}

# [6,13,10,11,9] pid: True for pid in [17, 27, 23, 10, 25, 11, 7, 6, 13]
inverse_servos = {pid: True for pid in [27, 23, 11, 7,13]}

initial_state, _, weight_matrix, _ = brain_params

weight_matrix = weight_matrix / np.max(np.abs(weight_matrix))
(
    cpg_network_structure,
    output_mapping,
) = active_hinges_to_cpg_network_structure_neighbor(hinges)
#initial_state = cpg_network_structure.make_uniform_state(0.5 * math.sqrt(2))

brain = BrainCpgNetworkStatic(
    initial_state=initial_state,
    weight_matrix=weight_matrix,
    output_mapping=output_mapping,
)

robot = ModularRobot(body=body, brain=brain)

In [46]:
config = Config(
    modular_robot=robot,
    hinge_mapping=hinge_mapping,
    initial_hinge_positions=initial_hinge_positions,
    run_duration=RUN_DURATION,
    control_frequency=CONTROL_FREQUENCY,
    inverse_servos=inverse_servos,
)

with open("lamarckism_conf.pickle", "wb") as f:
    pickle.dump(config, f)